### A pyiron workflow to calculate a grain boundary energy using multiple potentials, and compare to literature DFT data.

This notebook __doesn't__ work out of the box if you haven't configured LAMMPS for pyiron

conda install -c conda-forge lammps 

OR

mamba install -c conda-forge lammps

In the python environment in which you are running pyiron

WARNING: In practice/production you should configure a compiled LAMMPS executable that is optimised for your cluster!

In [1]:
from pyiron.project import Project

import numpy as np
import pandas as pd

from ase.lattice.cubic import BodyCenteredCubic as bcc
from ase.build import stack

from pyiron.atomistics.structure.atoms import ase_to_pyiron

In [2]:
pr = Project("GB_calcs")

Create the structure that we're going to use:
a $\Sigma 3 [1 \bar{1}0](111)$ Fe GB

In [3]:
surface1 = [1, 1, 1]
surface2 = [1, 1, -1]
rotation_axis = [1, -1, 0]
element = "Fe"
lc = 2.8318488966083
GB_name = "S3-RA110-S1-11"

# The minimum required length of the cell.
req_length = 15

v1 = list(-np.cross(rotation_axis,surface1))
v2 = list(-np.cross(rotation_axis,surface2)) 

length = 0
n = 0
while length < req_length:
    n += 1
    
    slab1 = bcc(symbol=element, latticeconstant=lc,directions=[rotation_axis,v1,surface1], size=[1,1,n])
    slab2 = bcc(symbol=element, latticeconstant=lc,directions=[rotation_axis,v2,surface2], size=[1,1,n])

    gb = stack(slab1, slab2)
    slab = stack(slab1, slab1)
    length = gb.cell[-1,-1]
    
    # Rattle the gb structure using rattle in ASE to perturb symmetry
    gb.rattle()
    slab.rattle()

Create the pyiron job, assign the structure to the job.

In [4]:
job = pr.create_job(job_type=pr.job_type.Lammps, job_name='Fe_S3_GB')

job.structure = ase_to_pyiron(gb)

We need to see the potentials, pyiron has an inbuilt potential database

It automatically finds the applicable potentials given the structure provided.

In [5]:
job.list_potentials()

['1997--Ackland-G-J--Fe--LAMMPS--ipr1',
 '1998--Meyer-R--Fe--LAMMPS--ipr1',
 '2001--Lee-B-J--Fe--LAMMPS--ipr1',
 '2001--Lee-B-J--Fe-Cr--LAMMPS--ipr1',
 '2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3',
 '2003--Mendelev-M-I--Fe-5--LAMMPS--ipr1',
 '2004--Ackland-G-J--Fe-P--LAMMPS--ipr1',
 '2004--Zhou-X-W--Fe--LAMMPS--ipr2',
 '2005--Lee-B-J--Fe-Cu--LAMMPS--ipr1',
 '2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1',
 '2006--Chamati-H--Fe--LAMMPS--ipr1',
 '2006--Kim-J--Fe-Pt--LAMMPS--ipr1',
 '2006--Lee-B-J--Fe-C--LAMMPS--ipr1',
 '2006--Lee-B-J--Fe-N--LAMMPS--ipr1',
 '2007--Lee-B-J--Fe-H--LAMMPS--ipr1',
 '2007--Mendelev-M-I--V-Fe--LAMMPS--ipr1',
 '2008--Hepburn-D-J--Fe-C--LAMMPS--ipr1',
 '2008--Sa-I--Fe-Nb--LAMMPS--ipr1',
 '2008--Sa-I--Fe-Ti--LAMMPS--ipr1',
 '2009--Bonny-G--Fe-Cu-Ni--LAMMPS--ipr1',
 '2009--Bonny-G--Fe-Ni--LAMMPS--ipr1',
 '2009--Kim-H-K--Fe-Ti-C--LAMMPS--ipr2',
 '2009--Kim-Y-M--Fe-Mn--LAMMPS--ipr1',
 '2009--Olsson-P-A-T--Fe--LAMMPS--ipr1',
 '2009--Stukowski-A--Fe-Cr--LAMMPS--ipr1',
 '2010--Ki

In [6]:
potentials_tested = job.list_potentials()
GB_energy_list = []
for potential in potentials_tested:
    try:
        job_GB_name = f"S3_GB_{potential}"
        job_GB = pr.create_job(job_type=pr.job_type.Lammps, job_name=job_GB_name)
        job_GB.structure = ase_to_pyiron(gb)
        job_GB.potential = potential
        job_GB.calc_minimize(pressure=None,
                            max_iter=10000)
        job_GB.run(delete_existing_job=True)
        
        job_SLAB_name = f"S3_SLAB_{potential}"
        job_SLAB = pr.create_job(job_type=pr.job_type.Lammps, job_name=job_SLAB_name)
        job_SLAB.structure = ase_to_pyiron(slab)
        job_SLAB.potential = potential
        job_SLAB.calc_minimize(pressure=None,
                            max_iter=10000)
        job_SLAB.run(delete_existing_job=True)
        
        # Formula for GB energy is E_GB_tot - E_SLAB_tot / 2 * (Interface area)
        # The 2 in the denominator is to account for the second interface present in the cell due to the periodic conditions
        
        GB_energy = (job_GB['output/generic/energy_tot'][-1] - job_SLAB['output/generic/energy_tot'][-1]) \
                    / (2 * job_GB.structure.cell.volume / job_GB.structure.cell[-1,-1]) * 16.02
        
        GB_energy_list.append(GB_energy)
    except:
        GB_energy_list.append(np.nan)

The job S3_GB_1997mmAcklandmGmJmmFemmLAMMPSmmipr1 was saved and received the ID: 323
The job S3_SLAB_1997mmAcklandmGmJmmFemmLAMMPSmmipr1 was saved and received the ID: 324
The job S3_GB_1998mmMeyermRmmFemmLAMMPSmmipr1 was saved and received the ID: 325
The job S3_SLAB_1998mmMeyermRmmFemmLAMMPSmmipr1 was saved and received the ID: 326
The job S3_GB_2001mmLeemBmJmmFemmLAMMPSmmipr1 was saved and received the ID: 327
The job S3_SLAB_2001mmLeemBmJmmFemmLAMMPSmmipr1 was saved and received the ID: 328
The job S3_GB_2001mmLeemBmJmmFemCrmmLAMMPSmmipr1 was saved and received the ID: 329
The job S3_SLAB_2001mmLeemBmJmmFemCrmmLAMMPSmmipr1 was saved and received the ID: 330
The job S3_GB_2003mmMendelevmMmImmFem2mmLAMMPSmmipr3 was saved and received the ID: 331
The job S3_SLAB_2003mmMendelevmMmImmFem2mmLAMMPSmmipr3 was saved and received the ID: 332
The job S3_GB_2003mmMendelevmMmImmFem5mmLAMMPSmmipr1 was saved and received the ID: 333
The job S3_SLAB_2003mmMendelevmMmImmFem5mmLAMMPSmmipr1 was saved

2023-08-11 00:32:39,466 - pyiron_log - WARNING - Job aborted - please remove it and run again! S3_GB_2011mmBonnymGmmFemCrmmLAMMPSmmipr2
2023-08-11 00:32:39,779 - pyiron_log - WARNING - Job aborted
2023-08-11 00:32:39,780 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 19.619625)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
ERROR on proc 0: Not a valid floating-point number: 'Infinity' (src/MANYBODY/pair_eam_alloy.cpp:174)
Last command: pair_coeff * * eam/alloy FeCr_d.eam.alloy Fe Cr



The job S3_GB_2011mmBonnymGmmFemCrmmLAMMPSmmipr2 was saved and received the ID: 379
The job S3_GB_2011mmBonnymGmmFemCrmmLAMMPSmmipr3 was saved and received the ID: 380
The job S3_SLAB_2011mmBonnymGmmFemCrmmLAMMPSmmipr3 was saved and received the ID: 381


2023-08-11 00:32:42,287 - pyiron_log - WARNING - Job aborted - please remove it and run again! S3_GB_2011mmBonnymGmmFemNimCrmmLAMMPSmmipr1
2023-08-11 00:32:42,537 - pyiron_log - WARNING - Job aborted
2023-08-11 00:32:42,538 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 19.619625)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
ERROR on proc 0: Not a valid floating-point number: 'INF' (src/MANYBODY/pair_eam_alloy.cpp:174)
Last command: pair_coeff * * FeNiCr.eam.alloy Fe Ni Cr



The job S3_GB_2011mmBonnymGmmFemNimCrmmLAMMPSmmipr1 was saved and received the ID: 382
The job S3_GB_2011mmBonnymGmmFemNimCrmmLAMMPSmmipr2 was saved and received the ID: 383
The job S3_SLAB_2011mmBonnymGmmFemNimCrmmLAMMPSmmipr2 was saved and received the ID: 384
The job S3_GB_2011mmChiesamSmmFem33mmLAMMPSmmipr1 was saved and received the ID: 385
The job S3_SLAB_2011mmChiesamSmmFem33mmLAMMPSmmipr1 was saved and received the ID: 386


...

Exception ignored in: <function FileHDFio.__del__ at 0x7f485597edd0>
Traceback (most recent call last):
  File "/root/github_pyiron/pyiron_base/pyiron_base/storage/hdfio.py", line 877, in __del__
    del self._file_name
AttributeError: _file_name


The job S3_GB_2020mmGrogermRmmComCrmFemMnmNimmLAMMPSmmipr1 was saved and received the ID: 430


Exception ignored in: <function FileHDFio.__del__ at 0x7f485597edd0>
Traceback (most recent call last):
  File "/root/github_pyiron/pyiron_base/pyiron_base/storage/hdfio.py", line 877, in __del__
    del self._file_name
AttributeError: _file_name
2023-08-11 00:33:19,020 - pyiron_log - WARNING - Job aborted - please remove it and run again! S3_GB_2020mmMorimHmmFemmLAMMPSmmipr1
Exception ignored in: <function FileHDFio.__del__ at 0x7f485597edd0>
Traceback (most recent call last):
  File "/root/github_pyiron/pyiron_base/pyiron_base/storage/hdfio.py", line 877, in __del__
    del self._file_name
AttributeError: _file_name


The job S3_GB_2021mmStarikovmSmmFemmLAMMPSmmipr1 was saved and received the ID: 432
The job S3_SLAB_2021mmStarikovmSmmFemmLAMMPSmmipr1 was saved and received the ID: 433
The job S3_GB_2021mmStarikovmSmmFemmLAMMPSmmipr2 was saved and received the ID: 434
The job S3_SLAB_2021mmStarikovmSmmFemmLAMMPSmmipr2 was saved and received the ID: 435
The job S3_GB_2021mmWenmMmmFemHmmLAMMPSmmipr1 was saved and received the ID: 436
The job S3_SLAB_2021mmWenmMmmFemHmmLAMMPSmmipr1 was saved and received the ID: 437
The job S3_GB_2022mmMahatamAmmAlmFemmLAMMPSmmipr1 was saved and received the ID: 438
The job S3_SLAB_2022mmMahatamAmmAlmFemmLAMMPSmmipr1 was saved and received the ID: 439
The job S3_GB_2022mmStarikovmSmmFemCrmHmmLAMMPSmmipr1 was saved and received the ID: 440
The job S3_SLAB_2022mmStarikovmSmmFemCrmHmmLAMMPSmmipr1 was saved and received the ID: 441
The job S3_GB_2022mmSunmYmmFemmLAMMPSmmipr1 was saved and received the ID: 442
The job S3_SLAB_2022mmSunmYmmFemmLAMMPSmmipr1 was saved and recei

Exception ignored in: <function FileHDFio.__del__ at 0x7f485597edd0>
Traceback (most recent call last):
  File "/root/github_pyiron/pyiron_base/pyiron_base/storage/hdfio.py", line 877, in __del__
    del self._file_name
AttributeError: _file_name
Exception ignored in: <function FileHDFio.__del__ at 0x7f485597edd0>
Traceback (most recent call last):
  File "/root/github_pyiron/pyiron_base/pyiron_base/storage/hdfio.py", line 877, in __del__
    del self._file_name
AttributeError: _file_name
Exception ignored in: <function FileHDFio.__del__ at 0x7f485597edd0>
Traceback (most recent call last):
  File "/root/github_pyiron/pyiron_base/pyiron_base/storage/hdfio.py", line 877, in __del__
    del self._file_name
AttributeError: _file_name
Exception ignored in: <function FileHDFio.__del__ at 0x7f485597edd0>
Traceback (most recent call last):
  File "/root/github_pyiron/pyiron_base/pyiron_base/storage/hdfio.py", line 877, in __del__
    del self._file_name
AttributeError: _file_name
Exception ig

Construct a dataframe which contains the information about GB energy, and their errors with respect to a DFT computed value.

The GB energy is from the value computed in this study:

Mai, H.L., Cui, X.Y., Scheiber, D., Romaner, L. and Ringer, S.P., 2022. The segregation of transition metals to iron grain boundaries and their effects on cohesion. Acta materialia, 231, p.117902.

In [7]:
data = {'potential': potentials_tested, 'GB_energy': GB_energy_list}
df = pd.DataFrame(data)

GB_energy_DFT = 1.58
df["err_DFT"] = np.round(GB_energy_DFT - df["GB_energy"], 3)
df["rel_err_DFT"] = np.round((GB_energy_DFT - df["GB_energy"]) / GB_energy_DFT * 100, 0)
df["GB_energy"] = np.round(df.GB_energy.tolist(), 3)
df = df.dropna(subset="GB_energy")
df

,potential,GB_energy,err_DFT,rel_err_DFT
0,1997--Ackland-G-J--Fe--LAMMPS--ipr1,0.980,0.600,38.0
1,1998--Meyer-R--Fe--LAMMPS--ipr1,1.078,0.502,32.0
2,2001--Lee-B-J--Fe--LAMMPS--ipr1,1.349,0.231,15.0
3,2001--Lee-B-J--Fe-Cr--LAMMPS--ipr1,1.342,0.238,15.0
4,2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3,1.386,0.194,12.0
5,2003--Mendelev-M-I--Fe-5--LAMMPS--ipr1,1.356,0.224,14.0
6,2004--Ackland-G-J--Fe-P--LAMMPS--ipr1,1.404,0.176,11.0
7,2004--Zhou-X-W--Fe--LAMMPS--ipr2,0.824,0.756,48.0
8,2005--Lee-B-J--Fe-Cu--LAMMPS--ipr1,1.342,0.238,15.0
9,2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1,1.404,0.176,11.0


Order the potentials by the minimum error wrt. DFT!

Now we have a good idea of how to search for an empirical potential which best captures the GB energetics of a specific grain boundary.

You can see that there are some potentials that are very, very accurate at predicting this specific GB energy. 

For a real study you should consider not just a single GB, but validate it across multiple GBs and ideally the specific phenomena you want to study.

In [8]:
df.sort_values(by="err_DFT").head(10)

,potential,GB_energy,err_DFT,rel_err_DFT
66,2022--Starikov-S--Fe-Cr-H--LAMMPS--ipr1,1.609,-0.029,-2.0
27,2010--Malerba-L--Fe--LAMMPS--ipr1,1.549,0.031,2.0
40,2013--Henriksson-K-O-E--Fe-C--LAMMPS--ipr1,1.499,0.081,5.0
55,2019--Byggmastar-J--Fe-O--LAMMPS--ipr1,1.499,0.081,5.0
57,2020--Byggmastar-J--Fe--LAMMPS--ipr1,1.466,0.114,7.0
9,2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1,1.404,0.176,11.0
15,2007--Mendelev-M-I--V-Fe--LAMMPS--ipr1,1.404,0.176,11.0
16,2008--Hepburn-D-J--Fe-C--LAMMPS--ipr1,1.404,0.176,11.0
43,2015--Eich-S-M--Fe-Cr--LAMMPS--ipr1,1.404,0.176,11.0
6,2004--Ackland-G-J--Fe-P--LAMMPS--ipr1,1.404,0.176,11.0
